In [21]:
from thermo.chemical import Chemical

emissivity = 0
temperature = 1900 #kelvin

boltzmann_constant = 5.6703e-10 

test this is 

aluminium = Chemical('aluminium')

aluminium_vapor_pressure = aluminium.VaporPressure.calculate(2000,"BOILING_CRITICAL") # takes temp in K, returns pressure in Pa
                                                                                      # not accurate!
print(aluminium_vapor_pressure)

graphite = Chemical('carbon|solid')
graphite_vapor_pressure = graphite.VaporPressure.T_dependent_property(1900) # takes temp in K, returns pressure in Pa
print(graphite_vapor_pressure)

764.004116608


Exception: Chemical name not recognized

In [ ]:
#Boltzman power consumption


emissivity = 0
temperature = 1900 #kelvin
boltzmann_constant = 5.6703e-10


In [14]:
help(aluminium.VaporPressure)

Help on VaporPressure in module thermo.vapor_pressure object:

class VaporPressure(thermo.utils.TDependentProperty)
 |  Class for dealing with vapor pressure as a function of temperature.
 |  Consists of four coefficient-based methods and four data sources, one
 |  source of tabular information, four corresponding-states estimators,
 |  any provided equation of state, and the external library CoolProp.
 |  
 |  Parameters
 |  ----------
 |  Tb : float, optional
 |      Boiling point, [K]
 |  Tc : float, optional
 |      Critical temperature, [K]
 |  Pc : float, optional
 |      Critical pressure, [Pa]
 |  omega : float, optional
 |      Acentric factor, [-]
 |  CASRN : str, optional
 |      The CAS number of the chemical
 |  eos : object, optional
 |      Equation of State object after :obj:`thermo.eos.GCEOS`
 |  
 |  Notes
 |  -----
 |  To iterate over all methods, use the list stored in
 |  :obj:`vapor_pressure_methods`.
 |  
 |  **WAGNER_MCGARRY**:
 |      The Wagner 3,6 original mo

In [30]:
help(aluminium.VaporPressure)

Help on VaporPressure in module thermo.vapor_pressure object:

class VaporPressure(thermo.utils.TDependentProperty)
 |  Class for dealing with vapor pressure as a function of temperature.
 |  Consists of four coefficient-based methods and four data sources, one
 |  source of tabular information, four corresponding-states estimators,
 |  any provided equation of state, and the external library CoolProp.
 |  
 |  Parameters
 |  ----------
 |  Tb : float, optional
 |      Boiling point, [K]
 |  Tc : float, optional
 |      Critical temperature, [K]
 |  Pc : float, optional
 |      Critical pressure, [Pa]
 |  omega : float, optional
 |      Acentric factor, [-]
 |  CASRN : str, optional
 |      The CAS number of the chemical
 |  eos : object, optional
 |      Equation of State object after :obj:`thermo.eos.GCEOS`
 |  
 |  Notes
 |  -----
 |  To iterate over all methods, use the list stored in
 |  :obj:`vapor_pressure_methods`.
 |  
 |  **WAGNER_MCGARRY**:
 |      The Wagner 3,6 original mo